In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
# IN and Out variables for data 
dimension = 120
fps = 100 #frames per second
filters = 128

x = tf.placeholder(tf.float32, [None, dimension, fps])  #Inputs
y = tf.placeholder(tf.float32, [None, 2])   #Labels
keep_prob = tf.placeholder(dtype= tf.float32)

In [3]:
#convolution Layer - 1 
weights_conv1 = tf.get_variable( "weights_conv1", shape = [dimension, 3, 1, filters], 
                                  initializer = tf.contrib.layers.xavier_initializer())

bias_conv1 = tf.get_variable( "bias_conv1", shape = [filters],
                             initializer = tf.contrib.layers.xavier_initializer())

input_x = tf.reshape(x, [-1, dimension, fps, 1])
print(input_x)

#output
conv1 = tf.nn.conv2d( input_x, weights_conv1, strides=[1, 1, 1, 1], padding='VALID') 
conv1_out = tf.nn.sigmoid( conv1 + bias_conv1 )
conv1_drop = tf.nn.dropout(conv1_out, keep_prob)
print(conv1_out)

Tensor("Reshape:0", shape=(?, 120, 100, 1), dtype=float32)
Tensor("Sigmoid:0", shape=(?, 1, 98, 128), dtype=float32)


In [4]:
#convolution layer - 2

weights_conv2 = tf.get_variable( "weights_conv2", shape = [ 1, 3, filters, filters],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv2 = tf.get_variable("bias_conv2", shape = [filters],
                            initializer = tf.contrib.layers.xavier_initializer())

#output
conv2 = tf.nn.conv2d( conv1_drop, weights_conv2, strides = [1, 1, 1, 1], padding = 'VALID')
conv2_out = tf.nn.sigmoid( conv2 + bias_conv2)
conv2_drop = tf.nn.dropout(conv2_out, keep_prob)
print(conv2_drop)

Tensor("dropout_1/mul:0", shape=(?, 1, 96, 128), dtype=float32)


In [5]:
#convolution layer - 3

weights_conv3 = tf.get_variable( "weights_conv3", shape = [ 1, 3, filters, filters],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv3 = tf.get_variable("bias_conv3", shape = [filters],
                            initializer = tf.contrib.layers.xavier_initializer())

#output
conv3 = tf.nn.conv2d( conv2_drop, weights_conv3, strides = [1, 1, 1, 1], padding ='VALID')
conv3_out = tf.nn.sigmoid( conv3 + bias_conv3)
conv3_drop = tf.nn.dropout(conv3_out, keep_prob)
print(conv3_drop)

Tensor("dropout_2/mul:0", shape=(?, 1, 94, 128), dtype=float32)


In [6]:
#Max pooling layer

weights_conv4 = tf.get_variable( "weights_conv4", shape = [ 1, 2, filters, filters],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv4 = tf.get_variable("bias_conv4", shape = [filters],
                            initializer = tf.contrib.layers.xavier_initializer())

conv4 = tf.nn.conv2d( conv3_drop, weights_conv4, strides = [1, 2, 2, 1], padding = "VALID")
conv4_out = tf.nn.sigmoid( conv4 + bias_conv4 )
conv4_drop = tf.nn.dropout(conv4_out, keep_prob)
pool_out = conv4_drop
print(pool_out)

Tensor("dropout_3/mul:0", shape=(?, 1, 47, 128), dtype=float32)


In [7]:
weights_conv5 = tf.get_variable( "weights_conv5", shape = [ 1, 3, filters, filters],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv5 = tf.get_variable("bias_conv5", shape = [filters],
                            initializer = tf.contrib.layers.xavier_initializer())

conv5 = tf.nn.conv2d( pool_out, weights_conv5, strides = [1, 1, 1, 1], padding = "VALID")
conv5_out = tf.nn.sigmoid( conv5 + bias_conv5 )
conv5_drop = tf.nn.dropout(conv5_out, keep_prob)
conv5_drop

<tf.Tensor 'dropout_4/mul:0' shape=(?, 1, 45, 128) dtype=float32>

In [8]:
weights_conv6 = tf.get_variable( "weights_conv6", shape = [ 1, 3, filters, filters],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv6 = tf.get_variable("bias_conv6", shape = [filters],
                            initializer = tf.contrib.layers.xavier_initializer())

conv6 = tf.nn.conv2d( conv5_drop, weights_conv6, strides = [1, 1, 1, 1], padding = "VALID")
conv6_out = tf.nn.sigmoid( conv6 + bias_conv6 )
conv6_drop = tf.nn.dropout(conv6_out, keep_prob)
conv6_drop

<tf.Tensor 'dropout_5/mul:0' shape=(?, 1, 43, 128) dtype=float32>

In [9]:
weights_conv7 = tf.get_variable( "weights_conv7", shape = [ 1, 3, filters, filters],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv7 = tf.get_variable("bias_conv7", shape = [filters],
                            initializer = tf.contrib.layers.xavier_initializer())

conv7 = tf.nn.conv2d( conv6_drop, weights_conv7, strides = [1, 1, 1, 1], padding = "VALID")
conv7_out = tf.nn.sigmoid( conv7 + bias_conv7 )
conv7_drop = tf.nn.dropout( conv7_out, keep_prob)
conv7_drop

<tf.Tensor 'dropout_6/mul:0' shape=(?, 1, 41, 128) dtype=float32>

In [10]:
weights_conv8 = tf.get_variable( "weights_conv8", shape = [ 1, 1, filters, 2],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv8 = tf.get_variable("bias_conv8", shape = [2],
                            initializer = tf.contrib.layers.xavier_initializer())

conv8 = tf.nn.conv2d( conv7_drop, weights_conv8, strides = [1, 1, 1, 1], padding = "VALID")
conv8_out = tf.nn.sigmoid( conv8 + bias_conv8 )
conv8_drop = tf.nn.dropout( conv8_out, keep_prob)
conv8_drop

<tf.Tensor 'dropout_7/mul:0' shape=(?, 1, 41, 2) dtype=float32>

In [11]:
#Final output layer
weights_final = tf.get_variable( "weights_final", shape = [ 1, 41, 2, 2],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_final = tf.get_variable("bias_final", shape = [2],
                            initializer = tf.contrib.layers.xavier_initializer())

conv_final = tf.nn.conv2d( conv8_drop, weights_final, strides = [1, 41, 1, 1], padding = "VALID")
conv_final = tf.nn.sigmoid( conv_final + bias_final )
conv_final

<tf.Tensor 'Sigmoid_8:0' shape=(?, 1, 1, 2) dtype=float32>

In [12]:
final_output = tf.reshape(conv_final, shape = [-1, 2])

output = tf.nn.softmax(final_output)

In [13]:
# losses and optimizers

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = final_output))
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal( tf.argmax(output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean( tf.cast( correct_predictions, tf.float32 ))

In [14]:
#initializing tensors using sessoion
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=None)

features_path = "/home/speechlab/ASV/siva/IFCC/overlap_data_100_50"
development_path = "/home/speechlab/ASV/siva/IFCC/dev_overlap_data_100_50"
#evaluation_path = "E:\\Hemlata_CSL\\MFCC\\CNN_input_testing_eval"
model_path = "/home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/"
    
dev_sp_path ="/home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/dev"
#eva_sp_path = "E:\\Hemlata_CSL\\MFCC\\4_conv\\eval"

In [15]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [16]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    indexs = len(list(os.listdir(test_path)))//2
    
    for folders in range(indexs):
        features = np.load(os.path.join(test_path, "features"+str(folders)+".npy"))
        labels = np.load(os.path.join(test_path, "labels"+str(folders)+".npy"))
        patches = labels.shape[0]
        patch_prob = session.run(output, feed_dict = {x:features, keep_prob:1.0})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [17]:
print("start")
time.sleep(20)
print("complete")

start
complete


In [18]:
#Training and testing accuracy
session.run(init)

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
# if not os.path.exists(dev_sp_path):
#     os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    
for i in range(0,50):
    for j in range(1,batches+1):  #number of batches
        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))

        train_step.run(session = session,feed_dict={ x:features[:1024], y:labels[:1024], keep_prob:1.0})
        train_step.run(session = session,feed_dict={ x:features[1024:], y:labels[1024:], keep_prob:1.0})
        
    print("iteration   ",(i+1),"\n")
    #test_accuracy = accuracy.eval(session = session, feed_dict = { x:test_features, y:test_labels, keep_prob:1.0 })
    #print("iteration   ",(i+1),"   dev_accuracy    ",test_accuracy)    
    #print()
    save_path = saver.save(session, model_path +str(i))
    

iteration    1 

INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/0 is not in all_model_checkpoint_paths. Manually adding it.
iteration    2 

INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/1 is not in all_model_checkpoint_paths. Manually adding it.
iteration    3 

INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/2 is not in all_model_checkpoint_paths. Manually adding it.
iteration    4 

INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/3 is not in all_model_checkpoint_paths. Manually adding it.
iteration    5 

INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/4 is not in all_model_checkpoint_paths. Manually adding it.
iteration    6 

INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/5 is not in all_model_checkpoint_paths. Manually adding it.
iteration    7 

INFO:tensorflow:/home/speechlab/ASV/siva/IFCC/100_50_full_c

In [19]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

with tf.Session(config=config) as session:
    for i in range(1,50):
        saver.restore(session, model_path +str(i))
        print("iteration  ",(i+1))
        eer_dev = get_EER(development_path, (i+1), dev_sp_path)
        print(" develop eer  ",eer_dev)
        print()
        

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/1
iteration   2
FRR   0.6911111111111111  FAR    0.696954732510288    mid    -0.9293453594048817
 develop eer   0.6911111111111111

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/2
iteration   3
FRR   0.6698148148148149  FAR    0.6983539094650206    mid    -0.9633228128606623
 develop eer   0.6698148148148149

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/3
iteration   4
FRR   0.0  FAR    1.0    mid    -0.9775620400905609
 develop eer   0.0

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/4
iteration   5
FRR   0.0  FAR    1.0    mid    -0.9850176572799683
 develop eer   0.0

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/5
iteration   6
FRR   0.0  FAR    1.0    mid

iteration   44
FRR   0.0  FAR    1.0    mid    -0.9999974071979523
 develop eer   0.0

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/44
iteration   45
FRR   0.0  FAR    1.0    mid    -0.9999979734420776
 develop eer   0.0

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/45
iteration   46
FRR   0.0  FAR    1.0    mid    -0.9999983906745911
 develop eer   0.0

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/46
iteration   47
FRR   0.0  FAR    1.0    mid    -0.9999985992908478
 develop eer   0.0

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/47
iteration   48
FRR   0.0  FAR    1.0    mid    -0.999998927116394
 develop eer   0.0

INFO:tensorflow:Restoring parameters from /home/speechlab/ASV/siva/IFCC/100_50_full_conv_3k_sig/Models/48
iteration   49
FRR   0.0  FAR    1.0

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

with tf.Session(config=config) as session:
    for i in range(0,100):
        saver.restore(session, model_path +str(i))
        print("iteration  ",(i+1))
        eer_eval = get_EER(evaluation_path, (i+1), eva_sp_path)
        print(" evaluat eer  ",eer_eval)
        print()